In [1]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight

# Create a grid map
grid_map = GridMap(width=10, height=10)

# Create some game entities
player = GameEntity(name="Player")
wall1 = GameEntity(name="Wall1", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
wall2 = GameEntity(name="Wall2", blocks_movement=BlocksMovement(value=True))
light_source = GameEntity(name="LightSource", blocks_light=BlocksLight(value=True))

# Place entities on the grid map
grid_map.get_node((2, 2)).add_entity(player)
grid_map.get_node((4, 4)).add_entity(wall1)
grid_map.get_node((6, 6)).add_entity(wall2)
grid_map.get_node((8, 8)).add_entity(light_source)

# Print the grid map
print("Grid Map:")
grid_map.print_grid()

# Get movement and vision arrays
movement_array = grid_map.get_movement_array()
vision_array = grid_map.get_vision_array()

print("\nMovement Array:")
for row in movement_array:
    print(row)

print("\nVision Array:")
for row in vision_array:
    print(row)

# Get nodes in radius
radius = 2
center_position = (2, 2)
nodes_in_radius = grid_map.get_nodes_in_radius(center_position, radius)

print(f"\nNodes in radius {radius} from position {center_position}:")
for node in nodes_in_radius:
    print(f"Node at position {node.position.value}")

Grid Map:
. . . . . . . . . . 
. . . . . . . . . . 
. . E . . . . . . . 
. . . . . . . . . . 
. . . . # . . . . . 
. . . . . . . . . . 
. . . . . . # . . . 
. . . . . . . . . . 
. . . . . . . . E . 
. . . . . . . . . . 

Movement Array:
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, False, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, False, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]

Vision Array:
[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]
[True, True, T

In [2]:
grid_map.get_node((2, 2))

Node(name='', id='7b9c8f71-9b71-4239-ac3a-c6120f647b75', position=Position(name='Position', id='8c30f01c-c296-411e-832b-e7fb93e490ef', value=(2, 2)), entities=[GameEntity(name=Player, id=e5c520cd-2ec0-4fe7-9fd2-339c394a0039, blocks_movement=name='BlocksMovement' id='aaf8661e-0191-4cf9-ba06-e620b8be00e7' value=False, blocks_light=name='BlocksLight' id='72a897dc-5e0d-4b7c-8fd5-1a26d7ad022d' value=False, node=Node(id=7b9c8f71-9b71-4239-ac3a-c6120f647b75, position=(2, 2)))], grid_map=<abstractions.goap.spatial.GridMap object at 0x00000146EA7E47F0>)